<a href="https://colab.research.google.com/github/HcodeQ/AI-DiabeteTrack---web-app-with-AI-features/blob/master/youtube_script_loader_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community langchain-together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 371.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import YoutubeLoader

In [ ]:
!pip install --upgrade pytube youtube-transcript-api


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.4 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders.youtube import TranscriptFormat

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=LgGmaX-l5K8",
    add_video_info=False,
    transcript_format=TranscriptFormat.CHUNKS,
    chunk_size_seconds=4,
    language=["fr", "id"],
    translation="fr",
)
loader.load()

[Document(metadata={'source': 'https://www.youtube.com/watch?v=LgGmaX-l5K8&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}, page_content='Vous pouvez tous dire la même chose si'),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=LgGmaX-l5K8&t=4s', 'start_seconds': 4, 'start_timestamp': '00:00:04'}, page_content="vous en avez envie. Moi personnellement les gars, j'en ai rien à faire et je pense exactement la même chose que Ancy"),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=LgGmaX-l5K8&t=8s', 'start_seconds': 8, 'start_timestamp': '00:00:08'}, page_content='flick. Certaines décisions arbitrales ont été injuste. Je lui ai dit ce que je'),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=LgGmaX-l5K8&t=12s', 'start_seconds': 12, 'start_timestamp': '00:00:12'}, page_content="pensais, je ne le dis pas ici. Et Arao qui dit il y a faute sur Martine. Et il rajoute l'arbitre à influencer le"),
 Document(metadata={'source': 'https://www.you

In [ ]:
!pip install langchain_groq langchain_core langchain-together langchain_community youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.3 MB/s eta 0:00:00


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat
from langchain_together import ChatTogether
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, field_validator
import re
from typing import List, Dict


TOGETHER_API_KEY = "...."

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    temperature=1,
    max_tokens=2000,
    api_key=TOGETHER_API_KEY
)

  #définition d'une structure de données
class yt_description(BaseModel):
    title: str = Field(description="video description title")
    resume: str = Field(description="video description resume")
    links: List[dict] = Field(description="video description links")
    timestamps: str = Field(description="video description timestamps")
    keywords: str = Field(description="video description keywords")
    call_to_action: str = Field(description="video description hashtags")
    about_channel: str = Field(description="video description about channel")


    @field_validator("keywords")
    @classmethod
    def keywords_must_be_list(cls, field):
        if not isinstance(field, str) or "," not in field:
            raise ValueError("Les mots clés doivent être séparés par des virgules et être une string.")
        return field

    @field_validator("links")
    @classmethod
    def links_in_correct_format(cls, field):
        link_pattern = re.compile(r"^[^:]+ : https?://[^\s]+$")
        for link_dict in field:
            if not isinstance(link_dict, dict):
                raise ValueError("les liens doivent être dans un format de dictionnaire.")
            if len(link_dict) != 1:
                raise ValueError("les liens ne continennent qu'un seul élément.")
        return field


def process_video():
    data_test = {
        "video_type": "Tutoriel",
        "video_url": "https://youtu.be/LGC9hI6QrJc",
        "description_tone": "based on the transcript tone",
        "optional_keywords": "SEO, IA, LangChain",
        "transcript_format": "CHUNKS",
        "transcript": "",
        "languages": ["fr", "en"],
        "translation": "fr",
        "hashtags": "#SEO #LangChain",
        "timestamps_mode": "automatique",
        "manual_timestamps": "",
        "useful_links": [{"Github": "https://github.com/"}, {"LangChain Docs": "https://python.langchain.com"}]
    }

    if data_test["timestamps_mode"] == "manuel":
        timestamps_instruction = f"Utiliser ces timestamps : {data_test['manual_timestamps']}"
    else:
        timestamps_instruction = "Génère automatiquement 4 à 7 timestamps clés avec des titres courts et pertinents."

    data_test["timestamps_instruction"] = timestamps_instruction

    # Chargement de la transcription
    loader = YoutubeLoader.from_youtube_url(
        data_test['video_url'],
        add_video_info=False,
        transcript_format=TranscriptFormat.CHUNKS,
        chunk_size_seconds=4,
        language=["fr", "id"],
        translation="fr",
    )
    docs = loader.load()

    # Nettoyage de la sortie pour garder uniquement les timecodes et le texte
    transcript = "\n".join([f"{chunk.page_content} - {chunk.metadata['start_timestamp']}" for chunk in docs])
    data_test['transcript'] = transcript

    #création d'un parseur de données
    parser = PydanticOutputParser(pydantic_object=yt_description)

    # Définition du prompt
    prompt = PromptTemplate(
        input_variables=[
            "video_type",
            "video_url",
            "description_tone",
            "optional_keywords",
            "transcript_format",
            "transcript",
            "languages",
            "translation",
            "hashtags",
            "timestamps_mode",
            "timestamps_instruction",
            "useful_links",
        ],
        partial_variables={"format_instructions": parser.get_format_instructions()},
        template="""
Tu es un expert YouTube et spécialiste SEO.

Ta mission est de produire une description de vidéo YouTube optimisée pour :
- attirer l’attention dès les premières secondes
- booster le SEO avec les bons mots-clés et hashtags
- encourager les likes, commentaires et abonnements
- s’adapter au type de vidéo ({video_type}) et au ton demandé ({description_tone})

Tu dois t’appuyer sur :
- la transcription suivante au format {transcript_format} : {transcript}
- la langue principale : {languages}
- la traduction éventuelle : {translation}
- les mots-clés supplémentaires (optionnels) : {optional_keywords}
- les hashtags suggérés : {hashtags}
- les indications pour le chapitrage : {timestamps_instruction}
- les liens utiles fournis : {useful_links}

---

🎯 **Consignes de sortie** :
Ne retourne rien d'autre que ce JSON avec les champs suivants :
1. "title" (1–2 phrases)
   → Adresse-toi directement au spectateur en utilisant un ton {description_tone}.
   **Exemple** :
   Tu veux découvrir comment j'ai transformé ma routine quotidienne pour booster ma productivité ? Regarde cette vidéo jusqu'au bout !

2. "resume"
   → Résume en quelques phrases ce que le spectateur va apprendre ou découvrir dans la vidéo. Sois personnel et engageant.
   **Exemple** :
   Dans cette vidéo, je partage avec toi mes astuces pour organiser efficacement ta journée et atteindre tes objectifs plus rapidement.

3. "links" (liste avec titre + lien) à partir des données d'entrée. Utilise les liens fournis dans la variable {useful_links} sous ce format :
   Exemple :
   🔗 Liens utiles
   Mon planner préféré : https://monplanner.com
   Application de gestion du temps : https://gestiondutemps.app

4. "timestamps"
   → Génére une section chapitrée en texte (pas en tableau), suivant les consignes suivantes : {timestamps_instruction}
   Utilise le mode de timestamps : {timestamps_mode}
   **Exemple** :
   ⏱️Chapitrages :
   00:00 – Introduction
   01:30 – Astuce n°1 : Planification
   03:45 – Astuce n°2 : Gestion du temps
   06:10 – Astuce n°3 : Élimination des distractions
   08:25 – Conclusion

5.  "keywords"
   → Intègre les mots-clés ({optional_keywords}) et hashtags ({hashtags}) de manière naturelle dans la description.
   **Exemple** :
   Mots-clés : productivité, organisation, gestion du temps
   #Productivité #Organisation #GestionDuTemps

6. "call_to_action"
   → Encourage le spectateur à liker, commenter, s’abonner et partager la vidéo.
   **Exemple** :
   Si tu as aimé cette vidéo, n'oublie pas de liker 👍, de t'abonner 🔔 et de partager avec tes amis ! Dis-moi en commentaire quelle astuce tu vas essayer en premier.

7. "about_channel"
   → Ajoute une brève présentation de la chaîne et invite à explorer d’autres contenus.

---

🎙️ Tu peux t’inspirer du style de description utilisé par les meilleurs créateurs de contenu YouTube dans ta thématique. Rends chaque section fluide, humaine et cohérente.

🎯 L’objectif est de produire une description complète, engageante et parfaitement optimisée à partir de la vidéo suivante : {video_url}

N'oublie pas d'utiliser des **pronoms personnels** ("je", "tu", "nous") pour créer une relation directe avec le spectateur.

""",

    )

    # Formatage du prompt
    #formatted_prompt = prompt.format(**data_test)

    # Passage de la requête au LLM
    #response = llm.invoke([("human", formatted_prompt)])
    #return response.content

    #création d'une chain
    chain = prompt | llm | parser
    result = chain.invoke(data_test)
    return result

appel = process_video()
print(appel)



title='Découvre le secret pour changer ta vie en ajustant ton filtre mental !' resume='Dans cette vidéo, je partagerai avec toi mes découvertes sur le fonctionnement de notre cerveau et comment notre perception du monde peut être influencée par un filtre subjectif. Tu apprendras comment ajuster ce filtre pour améliorer la qualité de ta vie et atteindre tes objectifs.' links=[{'Github': 'https://github.com/'}, {'LangChain Docs': 'https://python.langchain.com'}] timestamps="⏱️ Chapitrages : \n00:00 – Introduction \n02:00 – Le système d'activation réticulaire (SAR) \n04:30 – Comment notre perception du monde est influencée \n07:00 – Les deux problèmes majeurs liés à notre filtre mental \n09:30 – Comment ajuster ton filtre mental pour améliorer ta vie \n11:30 – Conclusion" keywords='Mots-clés : perception, cerveau, filtre mental, SEO, IA, LangChain. #SEO #LangChain #Perception #Cerveau' call_to_action="Si tu as aimé cette vidéo, n'oublie pas de liker, de t'abonner et de partager avec tes a

In [ ]:
!pip install fastapi uvicorn

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Dict
from yt_description_module

Parsing step